In [9]:
import cv2
import time
import ipywidgets as widgets
import IPython.display as display

In [10]:
CONFIDENCE_THRESHOLD = 0.2
NMS_THRESHOLD = 0.4
COLORS = [(0, 255, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]

In [11]:
class_names = []
with open("coco.names", "r") as f:
    class_names = [cname.strip() for cname in f.readlines()]

In [12]:
net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

In [13]:
model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)

In [14]:
path = '/home/kyj1103/'
#cap = cv2.VideoCapture(path+'/sejong_drive.mp4')
cap = cv2.VideoCapture(path+'/pedestrian10.mp4') 

frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('frame_size=',frame_size)

width, height = frame_size

frame_size= (1920, 1080)


In [15]:
wlmg = widgets.Image(layout = widgets.Layout(border='solid'))
display.display(wlmg)

Image(value=b'', layout="Layout(border='solid')")

In [16]:
# 비디오 객체 있을 시
if cap.isOpened():
    # 비디오 객체의 정보를 얻어옴
    ret,img = cap.read()
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        #동영상 파일에서 캡쳐된 이미지를 이밎 파일 스트림으로 다시 인코딩
        tmpStream = cv2.imencode('.jpeg',img)[1].tostring()
        wlmg.value = tmpStream
    
        # 20프레임이 되기 위한 딜레이 다만, 실제로 입력한 것보다 조금 더 딜레이가 있다.
        time.sleep(0.05)
        ret,img = cap.read()
    

        start = time.time()
        classes, scores, boxes = model.detect(frame, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
        end = time.time()

        start_drawing = time.time()
        for (classid, score, box) in zip(classes, scores, boxes):
            color = COLORS[int(classid) % len(COLORS)]
            label = "%s : %f" % (class_names[classid[0]], score)
            cv2.rectangle(img, box, color, 2)
            cv2.putText(img, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        
            # 블러처리
            left, top, width, height = box
            roi = img[top:top+height, left:left+width]   # 관심영역 지정
            roi = cv2.GaussianBlur(roi,(15,15),0)
            img[top:top+height, left:left+width] = roi   # 원본 이미지에 적용
        
        
        end_drawing = time.time()
    
        fps_label = "FPS: %.2f (excluding drawing time of %.2fms)" % (1 / (end - start), (end_drawing - start_drawing) * 1000)
        cv2.putText(img, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    
    
        
cap.release()
